In [93]:
import SimpleITK as sitk
import numpy as np

# define metrics

In [94]:
def get_accuracy(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    true_negatives = np.sum(np.logical_and(1-ground_truth, 1-predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))
    false_nagatives = np.sum(np.logical_and(ground_truth, 1-predicted))

    return (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_nagatives)

In [95]:
def get_precision(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))

    return (true_positives) / (true_positives + false_positives)

In [96]:
def get_recall(ground_truth, predicted):

    true_positives = np.sum(np.logical_and(ground_truth, predicted))
    false_nagatives = np.sum(np.logical_and(ground_truth, 1-predicted))

    return (true_positives) / (true_positives + false_nagatives)

In [97]:
def get_specificity(ground_truth, predicted):
    true_negatives = np.sum(np.logical_and(1-ground_truth, 1-predicted))
    false_positives = np.sum(np.logical_and(1-ground_truth, predicted))

    return (true_negatives) / (true_negatives + false_positives)

In [98]:
def get_f1_score(ground_truth, predicted):

    precision = get_precision(ground_truth, predicted)
    recall = get_recall(ground_truth, predicted)

    return 2 * (precision * recall) / (precision + recall)

In [99]:
# def get_Jaccard(ground_truth, predicted):
#     intersection = np.sum(np.logical_and(ground_truth, predicted))
#     union = np.sum(np.logical_or(ground_truth, predicted))

#     return intersection / union

def get_Jaccard(ground_truth, predicted):
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return overlap_measures_filter.GetJaccardCoefficient()

In [100]:
def get_DSC(ground_truth, predicted):
    return get_f1_score(ground_truth, predicted)

In [101]:
def get_volume_similarity(ground_truth, predicted):
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return overlap_measures_filter.GetVolumeSimilarity()

In [102]:
def get_HD(ground_truth, predicted):
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(sitk.GetImageFromArray(ground_truth), sitk.GetImageFromArray(predicted))
    
    return hausdorff_distance_filter.GetHausdorffDistance()

In [110]:
import numpy as np
from scipy.ndimage import morphology

def get_ASSD(input1, input2, sampling=1, connectivity=1):
    input_1 = input1.copy()
    input_2 = input2.copy() 

    conn = morphology.generate_binary_structure(input_1.ndim, connectivity)

    S = input_1 - morphology.binary_erosion(input_1, conn)
    Sprime = input_2 - morphology.binary_erosion(input_2, conn)

    dta = morphology.distance_transform_edt(~S,sampling)
    dtb = morphology.distance_transform_edt(~Sprime,sampling)
    
    sds = np.concatenate([np.ravel(dta[Sprime!=0]), np.ravel(dtb[S!=0])])
       
    return sds.mean()

# Execute metrics on examples

In [104]:
ground_truth = np.array([[0,0,0,0,0,0],
                         [0,0,1,1,0,0],
                         [0,1,1,1,1,0],
                         [0,1,1,1,1,0],
                         [0,0,0,0,0,0],
                         [0,0,0,0,0,0]], dtype=np.int32)

# Predicted segmentation
predicted1 = np.array([[0,0,0,0,0,0],
                       [0,1,1,1,1,0],
                       [0,1,1,1,0,0],
                       [0,1,1,1,0,0],
                       [0,0,0,0,0,0],
                       [0,0,0,0,0,0]], dtype=np.int32)

predicted2 = np.array([[0,0,0,0,0,0],
                       [0,0,0,1,0,0],
                       [0,1,1,1,0,0],
                       [0,1,1,1,1,0],
                       [0,0,0,1,0,0],
                       [0,0,0,1,0,0]], dtype=np.int32)

predicted3 = np.array([[0,0,0,0,0,0],
                       [0,0,0,0,1,0],
                       [0,0,1,1,1,0],
                       [0,1,1,1,1,0],
                       [0,0,0,0,0,0],
                       [0,0,0,0,0,0]], dtype=np.int32)
                       

In [111]:
for prediction in [predicted1, predicted2, predicted3]:
    print("Accuracy: ", get_accuracy(ground_truth, prediction))
    print("Precision: ", get_precision(ground_truth, prediction))
    print("Recall: ", get_recall(ground_truth, prediction))
    print("Specificity: ", get_specificity(ground_truth, prediction))
    print("F1 Score: ", get_f1_score(ground_truth, prediction))
    print("Jaccard: ", get_Jaccard(ground_truth, prediction))
    print("DSC: ", get_DSC(ground_truth, prediction))
    print("Volume Similarity: ", get_volume_similarity(ground_truth, prediction))
    print("Hausdorff Distance: ", get_HD(ground_truth, prediction))
    print("Average Symmetric Surface Distance: ", get_ASSD(ground_truth, prediction))
    print("\n")

Accuracy:  0.8888888888888888
Precision:  0.8
Recall:  0.8
Specificity:  0.9230769230769231
F1 Score:  0.8000000000000002
Jaccard:  0.6666666666666666
DSC:  0.8000000000000002
Volume Similarity:  0.0
Hausdorff Distance:  1.0
Average Symmetric Surface Distance:  3.999452597123684


Accuracy:  0.8888888888888888
Precision:  0.8
Recall:  0.8
Specificity:  0.9230769230769231
F1 Score:  0.8000000000000002
Jaccard:  0.6666666666666666
DSC:  0.8000000000000002
Volume Similarity:  0.0
Hausdorff Distance:  2.0
Average Symmetric Surface Distance:  4.426801438009301


Accuracy:  0.8888888888888888
Precision:  0.875
Recall:  0.7
Specificity:  0.9615384615384616
F1 Score:  0.7777777777777777
Jaccard:  0.6363636363636364
DSC:  0.7777777777777777
Volume Similarity:  0.2222222222222222
Hausdorff Distance:  1.0
Average Symmetric Surface Distance:  4.401298477386291




/tmp/ipykernel_167891/352518071.py:8: DeprecationWarning: Please import `generate_binary_structure` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  conn = morphology.generate_binary_structure(input_1.ndim, connectivity)
/tmp/ipykernel_167891/352518071.py:10: DeprecationWarning: Please import `binary_erosion` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  S = input_1 - morphology.binary_erosion(input_1, conn)
/tmp/ipykernel_167891/352518071.py:11: DeprecationWarning: Please import `binary_erosion` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  Sprime = input_2 - morphology.binary_erosion(input_2, conn)
/tmp/ipykernel_167891/352518071.py:14: DeprecationWarning: Please import `distance_transform_edt` from the `scipy.ndimage` namespace; the `scipy.n